In [1]:
import pandas as pd

In [4]:
df2007 = pd.read_csv("rendimento_escolar/Dados Tratados/dadosTratados2007.csv")
df2007

,Ano,Região,UF,Código do Município,Nome do Município,Localização,Rede,Total_Abandono_EF2007,Total_Abandono_EM2007
0,2007,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Estadual,1.6,--
1,2007,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Municipal,6.6,--
2,2007,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Publico,6.2,--
3,2007,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Rural,Total,6.2,--
4,2007,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Total,Estadual,5.9,18.2
...,...,...,...,...,...,...,...,...,...
33326,2007,Centro_Oeste,DF,5300108,BRASILIA,Urbana,Estadual,2.2,10.4
33327,2007,Centro_Oeste,DF,5300108,BRASILIA,Urbana,Federal,0,0
33328,2007,Centro_Oeste,DF,5300108,BRASILIA,Urbana,Particular,0.1,0.1
33329,2007,Centro_Oeste,DF,5300108,BRASILIA,Urbana,Publico,2.2,10.2


In [2]:
matriculas_df = pd.read_csv("alunos_matriculados/matriculas_fundamental_medio_2007.csv")
matriculas_df

,CO_MUNICIPIO,NO_UF,NO_MUNICIPIO,QT_MAT_FUND_Privada_Urbana,QT_MAT_FUND_Pública_Rural,QT_MAT_FUND_Pública_Urbana,QT_MAT_FUND_Privada_Rural,QT_MAT_MED_Privada_Urbana,QT_MAT_MED_Pública_Rural,QT_MAT_MED_Pública_Urbana,QT_MAT_MED_Privada_Rural,Total_Fundamental,Total_Medio
0,1100015,Rondônia,Alta Floresta D'Oeste,56.0,2534.0,2753.0,0.0,0.0,0.0,1161.0,0.0,5343.0,1161.0
1,1100023,Rondônia,Ariquemes,1182.0,2673.0,13609.0,0.0,240.0,0.0,3454.0,0.0,17464.0,3694.0
2,1100031,Rondônia,Cabixi,0.0,51.0,1154.0,0.0,0.0,0.0,265.0,0.0,1205.0,265.0
3,1100049,Rondônia,Cacoal,1766.0,3348.0,9244.0,57.0,644.0,0.0,3731.0,162.0,14415.0,4537.0
4,1100056,Rondônia,Cerejeiras,69.0,581.0,2509.0,0.0,16.0,0.0,687.0,0.0,3159.0,703.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,5222005,Goiás,Vianópolis,49.0,499.0,1693.0,0.0,11.0,0.0,578.0,0.0,2241.0,589.0
5560,5222054,Goiás,Vicentinópolis,0.0,9.0,1208.0,0.0,0.0,0.0,336.0,0.0,1217.0,336.0
5561,5222203,Goiás,Vila Boa,0.0,66.0,1028.0,0.0,0.0,0.0,216.0,0.0,1094.0,216.0
5562,5222302,Goiás,Vila Propício,0.0,554.0,491.0,0.0,0.0,106.0,134.0,0.0,1045.0,240.0


#### filtragem de total Rural E Urbano

In [6]:
# Lista das colunas que você quer manter (adicione as suas)
colunas_selecionadas = [
    'Ano', 
    'Região', 
    'UF', 
    'Código do Município', 
    'Nome do Município',
    'Localização',
    'Rede',
    'Total_Abandono_EF2007', 
    'Total_Abandono_EM2007',
    
]

# Filtro + seleção de colunas em uma única operação
dados_tratados_df = df2007[
   (df2007['Localização'] == 'Total') & (df2007['Rede'] == 'Total')
][colunas_selecionadas].copy()

# Resetar índice (opcional)
dados_tratados_df = dados_tratados_df.reset_index(drop=True)

In [7]:
dados_tratados_df

,Ano,Região,UF,Código do Município,Nome do Município,Localização,Rede,Total_Abandono_EF2007,Total_Abandono_EM2007
0,2007,Norte,RO,1100015,ALTA FLORESTA D'OESTE,Total,Total,5.8,18.2
1,2007,Norte,RO,1100023,ARIQUEMES,Total,Total,3.6,11.3
2,2007,Norte,RO,1100031,CABIXI,Total,Total,0.6,6.3
3,2007,Norte,RO,1100049,CACOAL,Total,Total,3.3,9.3
4,2007,Norte,RO,1100056,CEREJEIRAS,Total,Total,3.1,12.8
...,...,...,...,...,...,...,...,...,...
2703,2007,Centro_Oeste,GO,5222005,VIANOPOLIS,Total,Total,3.1,15.1
2704,2007,Centro_Oeste,GO,5222054,VICENTINOPOLIS,Total,Total,5.1,14.3
2705,2007,Centro_Oeste,GO,5222203,VILA BOA,Total,Total,1.3,13.5
2706,2007,Centro_Oeste,GO,5222302,VILA PROPICIO,Total,Total,6.4,14.2


#### tornas as linhas RURAL e URBANA em colunas 
#### ainda não esta sendo usada

In [5]:

# Supondo que seu dataframe se chame df_filtrado
df_pivoted = df_filtrado.pivot(
    index=['Ano', 'Região', 'UF', 'Código do Município', 'Nome do Município', 'Rede'],
    columns='Localização',
    values=['Total_Abandono_EF2007', 'Total_Abandono_EM2007']
)

# Resetar o índice para transformar os níveis do multi-índice em colunas normais
df_pivoted = df_pivoted.reset_index()

# Renomear as colunas para ficarem mais claras
df_pivoted.columns = [
    'Ano', 'Região', 'UF', 'Código do Município', 'Nome do Município', 'Rede',
    'Total_Abandono_EF2007_Rural', 'Total_Abandono_EF2007_Urbana',
    'Total_Abandono_EM2007_Rural', 'Total_Abandono_EM2007_Urbana'
]

# Reordenar as colunas se necessário
df_pivoted = df_pivoted[[
    'Ano', 'Região', 'UF', 'Código do Município', 'Nome do Município', 'Rede',
    'Total_Abandono_EF2007_Rural', 'Total_Abandono_EF2007_Urbana',
    'Total_Abandono_EM2007_Rural', 'Total_Abandono_EM2007_Urbana'
]]

In [6]:
df_pivoted

,Ano,Região,UF,Código do Município,Nome do Município,Rede,Total_Abandono_EF2007_Rural,Total_Abandono_EF2007_Urbana,Total_Abandono_EM2007_Rural,Total_Abandono_EM2007_Urbana
0,2007,Centro_Oeste,DF,5300108,BRASILIA,Total,2.1,1.8,5.3,8
1,2007,Centro_Oeste,GO,5200050,ABADIA DE GOIAS,Total,2.4,8.9,--,25.6
2,2007,Centro_Oeste,GO,5200100,ABADIANIA,Total,3.1,5.7,0,18.4
3,2007,Centro_Oeste,GO,5200134,ACREUNA,Total,NaN,6.4,NaN,14.5
4,2007,Centro_Oeste,GO,5200159,ADELANDIA,Total,NaN,1.9,NaN,14
...,...,...,...,...,...,...,...,...,...,...
2703,2007,Norte,TO,1721208,TOCANTINOPOLIS,Total,17.8,1.6,25.6,18.6
2704,2007,Norte,TO,1721257,TUPIRAMA,Total,0,3.4,--,6.4
2705,2007,Norte,TO,1721307,TUPIRATINS,Total,3.8,2.1,--,9.2
2706,2007,Norte,TO,1722081,WANDERLANDIA,Total,5.3,0.9,--,11.3


#### mescla os dois dataframes para incluir as colunas Total_Rural e Total_Urbano da planilha matriculas_detalhadas

In [8]:
# Limpeza dos nomes das colunas para evitar problemas de formatação
matriculas_df.columns = matriculas_df.columns.str.strip()
dados_tratados_df.columns = dados_tratados_df.columns.str.strip()

# Converter o código do município em ambas as tabelas para garantir que eles são compatíveis
matriculas_df['CO_MUNICIPIO'] = matriculas_df['CO_MUNICIPIO'].astype(str)
dados_tratados_df['Código do Município'] = dados_tratados_df['Código do Município'].astype(str)

# Merge das tabelas com base no código do município
merged_df = pd.merge(matriculas_df, dados_tratados_df, left_on='CO_MUNICIPIO', right_on='Código do Município')

# Verificar se a junção foi bem-sucedida e visualizar as primeiras linhas
merged_df[['Ano','CO_MUNICIPIO','Nome do Município','Localização','Total_Fundamental','Total_Medio', 'Total_Abandono_EF2007', 'Total_Abandono_EM2007']]


,Ano,CO_MUNICIPIO,Nome do Município,Localização,Total_Fundamental,Total_Medio,Total_Abandono_EF2007,Total_Abandono_EM2007
0,2007,1100015,ALTA FLORESTA D'OESTE,Total,5343.0,1161.0,5.8,18.2
1,2007,1100023,ARIQUEMES,Total,17464.0,3694.0,3.6,11.3
2,2007,1100031,CABIXI,Total,1205.0,265.0,0.6,6.3
3,2007,1100049,CACOAL,Total,14415.0,4537.0,3.3,9.3
4,2007,1100056,CEREJEIRAS,Total,3159.0,703.0,3.1,12.8
...,...,...,...,...,...,...,...,...
2703,2007,5222005,VIANOPOLIS,Total,2241.0,589.0,3.1,15.1
2704,2007,5222054,VICENTINOPOLIS,Total,1217.0,336.0,5.1,14.3
2705,2007,5222203,VILA BOA,Total,1094.0,216.0,1.3,13.5
2706,2007,5222302,VILA PROPICIO,Total,1045.0,240.0,6.4,14.2


#### realiza a multiplicação 

In [16]:

# Verificar os tipos das colunas antes de realizar a multiplicação
print(merged_df[['Total_Fundamental','Total_Medio', 'Total_Abandono_EF2007', 'Total_Abandono_EM2007']].dtypes)

# Converter explicitamente para float, caso não estejam nesse formato
merged_df['Total_Fundamental'] = pd.to_numeric(merged_df['Total_Fundamental'], errors='coerce')
merged_df['Total_Medio'] = pd.to_numeric(merged_df['Total_Medio'], errors='coerce')
merged_df['Total_Abandono_EF2007'] = pd.to_numeric(merged_df['Total_Abandono_EF2007'], errors='coerce')
merged_df['Total_Abandono_EM2007'] = pd.to_numeric(merged_df['Total_Abandono_EM2007'], errors='coerce')

# Substituir NaN por 0 para evitar multiplicação com valores ausentes
merged_df['Total_Abandono_EM2007'] = merged_df['Total_Abandono_EM2007'].fillna(0)
merged_df['Total_Abandono_EF2007'] = merged_df['Total_Abandono_EF2007'].fillna(0)

# Realizar a multiplicação
merged_df['Resultado_Multiplicacao_EF2007'] = ((merged_df['Total_Fundamental'] * merged_df['Total_Abandono_EF2007'])/100).astype(int)

merged_df['Resultado_Multiplicacao_EM2007'] = ((merged_df['Total_Medio'] * merged_df['Total_Abandono_EM2007'])/100).astype(int)

# Exibir as primeiras linhas com o resultado da multiplicação
print(merged_df[['CO_MUNICIPIO', 'Nome do Município','Total_Fundamental','Total_Medio', 'Total_Abandono_EF2007', 'Total_Abandono_EM2007', 'Resultado_Multiplicacao_EF2007','Resultado_Multiplicacao_EM2007']].head())


Total_Fundamental        float64
Total_Medio              float64
Total_Abandono_EF2007    float64
Total_Abandono_EM2007    float64
dtype: object
  CO_MUNICIPIO      Nome do Município  Total_Fundamental  Total_Medio  \
0      1100015  ALTA FLORESTA D'OESTE             5343.0       1161.0   
1      1100023              ARIQUEMES            17464.0       3694.0   
2      1100031                 CABIXI             1205.0        265.0   
3      1100049                 CACOAL            14415.0       4537.0   
4      1100056             CEREJEIRAS             3159.0        703.0   

   Total_Abandono_EF2007  Total_Abandono_EM2007  \
0                    5.8                   18.2   
1                    3.6                   11.3   
2                    0.6                    6.3   
3                    3.3                    9.3   
4                    3.1                   12.8   

   Resultado_Multiplicacao_EF2007  Resultado_Multiplicacao_EM2007  
0                             309       

In [14]:
merged_df[['CO_MUNICIPIO', 'Nome do Município','Localização', 'Total_Fundamental','Total_Medio', 'Total_Abandono_EF2007', 'Total_Abandono_EM2007', 'Resultado_Multiplicacao_EF2007','Resultado_Multiplicacao_EM2007']].head()

,CO_MUNICIPIO,Nome do Município,Localização,Total_Fundamental,Total_Medio,Total_Abandono_EF2007,Total_Abandono_EM2007,Resultado_Multiplicacao_EF2007,Resultado_Multiplicacao_EM2007
0,1100015,ALTA FLORESTA D'OESTE,Total,5343.0,1161.0,5.8,18.2,309.894,211.302
1,1100023,ARIQUEMES,Total,17464.0,3694.0,3.6,11.3,628.704,417.422
2,1100031,CABIXI,Total,1205.0,265.0,0.6,6.3,7.230,16.695
3,1100049,CACOAL,Total,14415.0,4537.0,3.3,9.3,475.695,421.941
4,1100056,CEREJEIRAS,Total,3159.0,703.0,3.1,12.8,97.929,89.984
